In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()  # charge les variables d'environnement à partir du fichier .env

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

connection_url = f'postgresql://{username}:{password}@{host}:{port}/{database}'

engine = create_engine(connection_url)
try:
    connection = engine.connect()
    print("Connection successful")
except Exception as e:
    print(f"Connection failed: {e}")


# Chargement des données
athletes_df = pd.read_sql("SELECT * FROM olympic_athletes;", con=engine)
hosts_df = pd.read_sql("SELECT * FROM olympic_hosts;", con=engine)
medals_df = pd.read_sql("SELECT * FROM olympic_medals;", con=engine)
results_df = pd.read_sql("SELECT * FROM olympic_results;", con=engine)

# Affichage des premières lignes de chaque dataframe pour vérification
print("Premières lignes de olympic_athletes :\n", athletes_df.head())
print("Premières lignes de olympic_hosts :\n", hosts_df.head())
print("Premières lignes de olympic_medals :\n", medals_df.head())
print("Premières lignes de olympic_results :\n", results_df.head())


In [ ]:
country_renaming = {
        'Soviet Union': 'Russia',
        'Yugoslavia': 'Serbia',
        'Russian Federation': 'Russia',
        'German Democratic Republic (Germany)': 'Germany',
        'Federal Republic of Germany': 'Germany',
        'Czechoslovakia': 'Czech Republic',
        'Unified Team': 'Russia',
        'ROC': 'Russia',
        'Olympic Athletes from Russia': 'Russia',
        'United Arab Republic': 'Egypt',
        'West Indies Federation': 'Jamaica',
        'Bohemia': 'Czech Republic',
        'Korea Team': 'Korea',
        'Malaya': 'Malaysia',
        'Côte d\'Ivoire': 'Ivory Coast',
        'Democratic Republic of Timor-Leste': 'East Timor',
        'The Former Yugoslav Republic of Macedonia': 'North Macedonia' 
    }

def final_clean_country_names(df):

     # Nettoyage des anomalies 
    df['country_name'] = df['country_name'].replace(country_renaming)
    df['country_name'] = df['country_name'].str.replace(r".*https?://.*", "", regex=True)
    df['country_name'] = df['country_name'].str.strip()
    df['country_name'] = df['country_name'].replace({
        'None)]"': np.nan, '10': np.nan, '25': np.nan, '17': np.nan, '14': np.nan, '5': np.nan, '3': np.nan, '11': np.nan, '1': np.nan, '16': np.nan, '': np.nan, "Jr.'": np.nan, "CÃ´te d'Ivoire": "Ivory Coast"
    })

    return df

# Appliquer le nettoyage sur les DataFrames concernés
medals_df = final_clean_country_names(medals_df)
results_df = final_clean_country_names(results_df)

results_df['country_name'] = results_df['country_name'].replace(country_renaming)
medals_df['country_name'] = medals_df['country_name'].replace(country_renaming)

In [ ]:
print("\nNoms de pays uniques dans olympic_results après le nettoyage :")
print(results_df['country_name'].dropna().unique())

In [ ]:
results_df['game_year'] = results_df['slug_game'].str.extract(r'(\d{4})').astype(int)
results_df['game_year'].count()

In [ ]:
hosts_df.head().count()

In [ ]:
summer_games = hosts_df[hosts_df['game_season'] == 'Summer']
winter_games = hosts_df[hosts_df['game_season'] == 'Winter']

In [ ]:
print("\nAperçu des données des JO d'hiver (Winter Games):")
print(winter_games.head())

In [ ]:
print("\nAperçu des données des JO d'été (Summer Games):")
print(summer_games.head())

In [ ]:
results_df['game_year'] = results_df['slug_game'].str.extract(r'(\d{4})').astype(int)

# Vérifier les colonnes disponibles après extraction de l'année
print("\nColonnes disponibles dans results_df après extraction de l'année:")
print(results_df.columns)
print("\nPremières lignes de results_df après extraction de l'année:")
print(results_df.head())

In [ ]:
# Calcul du cumul de chaque type de médailles par JO par pays
medals_count = results_df.groupby(['country_name', 'game_year', 'medal_type']).size().unstack(fill_value=0).reset_index()
medals_count['total_medals'] = medals_count[['GOLD', 'SILVER', 'BRONZE']].sum(axis=1)

# Afficher un aperçu des cumul de médailles
print("\nAperçu du cumul de médailles par JO par pays:")
print(medals_count.head())


In [ ]:
sns.pairplot(results_df)
plt.show()


In [ ]:
# Calcul du nombre de disciplines et d'épreuves disputées par JO par pays
disciplines_count = results_df.groupby(['country_name', 'game_year']).agg(
    total_disciplines=pd.NamedAgg(column='discipline_title', aggfunc='nunique'),
    total_events=pd.NamedAgg(column='event_title', aggfunc='nunique')
).reset_index()

# Afficher un aperçu des disciplines et épreuves
print("\nAperçu du nombre de disciplines et d'épreuves disputées par JO par pays:")
print(disciplines_count.head())


In [ ]:
# Fusionner les datasets pour obtenir un dataset complet
historic_olympic_data = medals_count.merge(disciplines_count, on=['country_name', 'game_year'], how='left')

# Afficher un aperçu des données fusionnées
print("\nAperçu des données fusionnées (historic_olympic_data):")
print(historic_olympic_data.head())


In [ ]:
import plotly.express as px

# Analyse story telling : classement des pays par JO en termes de total de médailles gagnées
fig = px.bar(historic_olympic_data, 
             x="country_name", 
             y="total_medals", 
             animation_frame="game_year", 
             hover_name="country_name",
             category_orders={"game_year": sorted(historic_olympic_data['game_year'].unique())})

fig.update_layout(
    title="Country ranked on Total medals since 1896",
    xaxis_title="Country",
    yaxis_title="Total medals",
    showlegend=False)

fig.show()


In [ ]:
# Classement des 30 premiers pays par JO en termes de total de médailles gagnées
top30_countries = historic_olympic_data.groupby('country_name')['total_medals'].sum().nlargest(30).index
df_top30 = historic_olympic_data[historic_olympic_data['country_name'].isin(top30_countries)]

fig_top30 = px.bar(df_top30, 
                   x="country_name", 
                   y="total_medals", 
                   animation_frame="game_year", 
                   hover_name="country_name",
                   category_orders={"game_year": sorted(df_top30['game_year'].unique())})

fig_top30.update_layout(
    title="Top 30 countries on Total medals since 1896",
    xaxis_title="Country",
    yaxis_title="Total medals",
    showlegend=False)

fig_top30.show()

# Préparer les données nettoyées pour les sauvegarder dans la base de données
historic_olympic_data.to_sql('cleaned_olympic_data', con=engine, if_exists='replace', index=False)
print("Données nettoyées sauvegardées dans la base de données.")

In [ ]:
fig = px.scatter(historic_olympic_data, x='game_year', y='total_medals', color='country_name', 
                 size='total_events', hover_data=['total_disciplines'])
fig.update_layout(title='Total Medals Over Years by Country', xaxis_title='Year', yaxis_title='Total Medals')
fig.show()


In [ ]:
# Lire les données sauvegardées dans la table 'cleaned_olympic_data'
saved_data = pd.read_sql("SELECT * FROM cleaned_olympic_data;", con=engine)

# Afficher un aperçu des données sauvegardées
print("\nAperçu des données sauvegardées (cleaned_olympic_data):")
print(saved_data.head())


In [ ]:
results_df.value_unit.unique()

results_df = results_df.rename({'slug_game':'game_slug'},axis=1)
results_df.columns

In [ ]:
def categorize_value_unit(value):
    if pd.isna(value):
        return 'NaN'
    elif 'm' in value or 'cm' in value or 'km' in value:
        return 'Distance'
    elif 's' in value or 'min' in value or 'h' in value:
        return 'Time'
    elif 'pts' in value or 'points' in value:
        return 'Points'
    else:
        return 'Other'

# Appliquer la catégorisation sur 'value_unit'
results_df['value_unit_category'] = results_df['value_unit'].apply(categorize_value_unit)

valid_value_types = ['TIME', 'DISTANCE', 'POINTS', 'WEIGHT', 'RANK', 'SCORE', 'STROKES']
results_df['value_type'] = results_df['value_type'].apply(lambda x: x if x in valid_value_types else 'Other')

# Afficher les valeurs uniques dans les nouvelles colonnes
print("\nValeurs uniques dans 'value_unit_category':")
print(results_df['value_unit_category'].unique())

In [ ]:
# Sélectionner les fonctionnalités pertinentes
features = ['discipline_title', 'event_title', 'game_slug', 'participant_type', 'country_name', 'game_year', 'value_unit_category', 'value_type']
X = results_df[features]

In [ ]:
assert not historic_olympic_data.isnull().values.any(), "Il y a des valeurs manquantes dans le dataset fusionné."
assert all(col in results_df.columns for col in features), "Certaines colonnes attendues sont manquantes dans results_df."


In [ ]:
fig = px.scatter(historic_olympic_data, x='game_year', y='total_medals', color='country_name', 
                 size='total_events', hover_data=['total_disciplines'])
fig.update_layout(title='Total Medals Over Years by Country', xaxis_title='Year', yaxis_title='Total Medals')
fig.show()


In [ ]:
# Sélectionner la cible
y = results_df['medal_type']

In [ ]:
# Gérer les valeurs manquantes dans `medal_type`
results_df['medal_type'].fillna('NO_MEDAL', inplace=True)

In [ ]:
# Gérer les valeurs manquantes dans 'value_unit_category'
X.loc[:, 'value_unit_category'] = X['value_unit_category'].fillna('NaN')

In [ ]:
# print("Valeurs uniques de 'medal_type':")
# print(results_df['medal_type'].unique())

# results_df['medal_type'].fillna('NO_MEDAL', inplace=True)

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

# Pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['game_year']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['discipline_title', 'event_title', 'game_slug', 'participant_type', 'country_name', 'value_unit_category', 'value_type'])
    ])

# Pipeline de modèle
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Entraîner le modèle
model.fit(X_train, y_train)

# Prédire sur les données de test
y_pred = model.predict(X_test)

# Évaluer le modèle
print("\nRapport de classification:")
print(classification_report(y_test, y_pred))

# Exemple de prédiction
example = X_test.iloc[0:1]  # Prendre un DataFrame au lieu d'une série
print("\nExemple de prédiction pour l'entrée suivante:")
print(example)
print("Prédiction du modèle:", model.predict(example))

In [ ]:
# Graphiques
# 1. Répartition des types de médailles
plt.figure(figsize=(10, 6))
sns.countplot(data=results_df, x='medal_type', order=results_df['medal_type'].value_counts().index)
plt.title('Répartition des types de médailles')
plt.xlabel('Type de médaille')
plt.ylabel('Nombre')
plt.show()


In [ ]:
results_df['medal_type'].fillna('NO_MEDAL', inplace=True)

# Calcul du cumul de chaque type de médailles par JO par pays
medals_count = results_df.groupby(['country_name', 'game_year', 'medal_type']).size().unstack(fill_value=0).reset_index()
medals_count['total_medals'] = medals_count[['BRONZE', 'GOLD', 'SILVER', 'NO_MEDAL']].sum(axis=1)

# Obtenir la liste de tous les pays triés par nombre total de médailles
all_countries = results_df['country_name'].value_counts().index

# Diviser la liste des pays en groupes de 30
group_size = 30
country_groups = [all_countries[i:i + group_size] for i in range(0, len(all_countries), group_size)]

# Visualiser chaque groupe de pays
for i, group in enumerate(country_groups):
    group_medals_by_year_country = medals_count[medals_count['country_name'].isin(group)]
    plt.figure(figsize=(14, 8))
    sns.lineplot(data=group_medals_by_year_country, x='game_year', y='total_medals', hue='country_name')
    plt.title(f"Évolution du nombre total de médailles pour le groupe de pays {i+1}")
    plt.xlabel("Année des Jeux")
    plt.ylabel("Total des médailles")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Obtenir les noms des fonctionnalités après transformation
num_features = ['game_year']
cat_features = ['discipline_title', 'event_title', 'game_slug', 'participant_type', 'country_name', 'value_unit_category', 'value_type']

# Obtenir les noms des fonctionnalités après encodage
encoded_cat_features = model.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(cat_features)
all_features = np.concatenate([num_features, encoded_cat_features])

# Extraire les importances des fonctionnalités du modèle de forêt aléatoire
feature_importances = model.named_steps['classifier'].feature_importances_

# Vérifier les longueurs
print(f"Longueur des feature_names: {len(all_features)}")
print(f"Longueur des feature_importances: {len(feature_importances)}")

# Créer le DataFrame des importances des fonctionnalités
importance_df = pd.DataFrame({'Feature': all_features, 'Importance': feature_importances})

# Trier par importance et sélectionner les 20 plus importantes
top_importances = importance_df.sort_values(by='Importance', ascending=False).head(20)

# Visualiser les 20 importances des fonctionnalités les plus importantes
plt.figure(figsize=(12, 10))
sns.barplot(data=top_importances, x='Importance', y='Feature')
plt.title('Top 20 caractéristiques importantes dans le modèle de forêt aléatoire')
plt.xlabel('Importance')
plt.ylabel('Caractéristique')
plt.show()


In [ ]:
# # Préparer les données nettoyées pour les sauvegarder dans la base de données
# historic_olympic_data.to_sql('cleaned_olympic_data', con=engine, if_exists='replace', index=False)
# print("Données nettoyées sauvegardées dans la base de données.")

In [ ]:
# Charger les colonnes spécifiées de la table olympic_athletes
athletes_df = pd.read_sql("SELECT athlete_url, athlete_full_name, games_participations, first_game, athlete_medals FROM olympic_athletes;", con=engine)

# Afficher un aperçu des données chargées
print("Aperçu des données chargées d'athletes_df :")
print(athletes_df.head())

In [ ]:
# Remplacer les valeurs None dans 'athlete_medals' par 'NO_MEDAL'
athletes_df['athlete_medals'].fillna('NO_MEDAL', inplace=True)

# Afficher un aperçu des données après remplacement
print("Aperçu des données après remplacement des None dans 'athlete_medals' :")
print(athletes_df.head())

In [ ]:
# Afficher un aperçu des premières lignes de results_df
print("Aperçu de results_df:")
print(results_df.head())
print(results_df.columns)

# Afficher un aperçu des premières lignes de athletes_df
print("Aperçu de athletes_df:")
print(athletes_df.head())
print(athletes_df.columns)

In [ ]:
# Joindre les datasets results_df et athletes_df sur 'athlete_full_name'
combined_df = pd.merge(results_df, athletes_df, how='left', on='athlete_full_name')

# Afficher un aperçu des données combinées
print("Aperçu des données combinées (combined_df) :")
print(combined_df.head())

In [ ]:
# Sélectionner les fonctionnalités pertinentes
features = ['discipline_title', 'event_title', 'game_slug', 'participant_type', 'country_name', 
            'game_year', 'value_unit_category', 'value_type', 'athlete_medals', 
            'games_participations', 'first_game']

X = combined_df[features]

# Sélectionner la cible
y = combined_df['medal_type']


In [ ]:
# Afficher le nombre de valeurs manquantes par colonne
missing_values = combined_df.isnull().sum()
print("Nombre de valeurs manquantes par colonne:\n", missing_values)


In [ ]:
# Afficher les premières lignes des colonnes avec des valeurs manquantes
columns_with_na = combined_df.columns[combined_df.isnull().any()].tolist()
print("\nAperçu des colonnes avec des valeurs manquantes:")
for column in columns_with_na:
    print(f"\n{column} - Nombre de valeurs manquantes: {combined_df[column].isnull().sum()}")
    print(combined_df[column].unique())


In [ ]:
print("Valeurs uniques de 'athlete_medals':")
print(combined_df['athlete_medals'].unique())

In [ ]:
# Remplir les valeurs manquantes
combined_df['country_name'].fillna('Unknown', inplace=True)
combined_df['games_participations'].fillna(combined_df['games_participations'].median(), inplace=True)
combined_df['first_game'].fillna('Unknown', inplace=True)
combined_df['athlete_medals'].fillna('NO_MEDAL', inplace=True)

In [ ]:
# Traitement des valeurs dans 'athlete_medals'
def process_medals(medals):
    if medals == 'NO_MEDAL':
        return 0, 0, 0
    gold_medals = 0
    silver_medals = 0
    bronze_medals = 0
    i = 0
    while i < len(medals):
        if medals[i].isdigit():
            num_medals = int(medals[i])
            i += 1
            if i < len(medals) and medals[i] == 'G':
                gold_medals += num_medals
            elif i < len(medals) and medals[i] == 'S':
                silver_medals += num_medals
            elif i < len(medals) and medals[i] == 'B':
                bronze_medals += num_medals
            i += 1
        else:
            i += 1
    return gold_medals, silver_medals, bronze_medals

combined_df['gold_medals'], combined_df['silver_medals'], combined_df['bronze_medals'] = zip(*combined_df['athlete_medals'].apply(process_medals))
combined_df = combined_df.drop(columns='athlete_medals')

# Ajouter une colonne pour le nombre total de médailles remportées par chaque athlète
combined_df['total_medals'] = combined_df['gold_medals'] + combined_df['silver_medals'] + combined_df['bronze_medals']

# Afficher un aperçu des données après traitement des médailles
print(combined_df[['athlete_full_name', 'gold_medals', 'silver_medals', 'bronze_medals', 'total_medals']].head())


In [ ]:
# Trier le DataFrame par nombre total de médailles en ordre décroissant
top_athletes = combined_df.sort_values('total_medals', ascending=False)

# Afficher les 10 premiers athlètes ayant remporté le plus de médailles
print(top_athletes[['athlete_full_name', 'gold_medals', 'silver_medals', 'bronze_medals', 'total_medals']].head(10))


In [ ]:
print(top_athletes['athlete_full_name'].tolist())


In [ ]:
top_athletes_sorted = top_athletes[top_athletes['total_medals'] > 0].sort_values('total_medals', ascending=False)
print(top_athletes_sorted[['athlete_full_name', 'gold_medals', 'silver_medals', 'bronze_medals']].head(10))


In [ ]:
print("Nombre de lignes dans top_athletes :", len(top_athletes))


In [ ]:
print("Nombre d'athlètes ayant remporté au moins une médaille :", len(top_athletes[top_athletes['total_medals'] > 0]))


In [ ]:
# Visualiser la distribution des médailles parmi les athlètes
plt.figure(figsize=(10, 6))
sns.barplot(data=top_athletes.head(10), x='total_medals', y='athlete_full_name', hue='gold_medals', dodge=False)
plt.title('Top 10 Athletes by Total Medals')
plt.xlabel('Total Medals')
plt.ylabel('Athlete')
plt.show()

# Vérifier les résultats
print("Nombre de lignes dans top_athletes :", len(top_athletes))

In [ ]:
# Séparer les données de 2020 des autres années
data_2020 = combined_df[combined_df['game_year'] == 2020]
data_other_years = combined_df[combined_df['game_year'] != 2020]

# Afficher un aperçu des données de 2020
print("Aperçu des données de 2020:")
print(data_2020.head())

# Afficher un aperçu des données des autres années
print("Aperçu des données des autres années:")
print(data_other_years.head())


In [ ]:
# Sélectionner les fonctionnalités pertinentes pour l'entraînement du modèle
features = ['discipline_title', 'event_title', 'game_slug', 'participant_type', 'country_name', 
            'game_year', 'value_unit_category', 'value_type', 'gold_medals', 'silver_medals', 'bronze_medals', 
            'games_participations', 'first_game']

X_train = data_other_years[features]
y_train = data_other_years['medal_type']

# Gérer les valeurs manquantes dans les fonctionnalités
X_train['value_unit_category'].fillna('NaN', inplace=True)
X_train['first_game'].fillna('Unknown', inplace=True)

# Pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['game_year', 'games_participations', 'gold_medals', 'silver_medals', 'bronze_medals']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['discipline_title', 'event_title', 'game_slug', 
                                                         'participant_type', 'country_name', 'value_unit_category', 
                                                         'value_type', 'first_game'])
    ])

# Pipeline de modèle
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Entraîner le modèle sur les données des années précédentes
model.fit(X_train, y_train)


In [ ]:
# Préparer les données de 2020 pour la prédiction
X_2020 = data_2020[features]

# Gérer les valeurs manquantes dans les fonctionnalités
X_2020['value_unit_category'].fillna('NaN', inplace=True)
X_2020['first_game'].fillna('Unknown', inplace=True)

# Faire des prédictions sur les données de 2020
y_pred_2020 = model.predict(X_2020)

# Ajouter les prédictions au DataFrame de 2020
data_2020['predicted_medal_type'] = y_pred_2020

# Afficher un aperçu des prédictions de 2020
print("Aperçu des prédictions pour 2020:")
print(data_2020[['athlete_full_name', 'predicted_medal_type']].head())


In [ ]:
# Comparer les prédictions aux résultats réels
comparison = data_2020[['athlete_full_name', 'medal_type', 'predicted_medal_type']]

# Calculer la précision des prédictions
accuracy = (comparison['medal_type'] == comparison['predicted_medal_type']).mean()
print(f"Précision des prédictions pour 2020: {accuracy:.2f}")

# Afficher les prédictions incorrectes
incorrect_predictions = comparison[comparison['medal_type'] != comparison['predicted_medal_type']]
print("Prédictions incorrectes:")
print(incorrect_predictions.head())


In [ ]:
# Charger les données de la table olympic_athletes
athletes_df = pd.read_sql("SELECT athlete_url, athlete_full_name, games_participations, first_game, athlete_medals FROM olympic_athletes;", con=engine)

# Remplacer les valeurs None dans 'athlete_medals' par 'NO_MEDAL'
athletes_df['athlete_medals'].fillna('NO_MEDAL', inplace=True)

# Joindre les datasets results_df et athletes_df sur 'athlete_full_name'
combined_df = pd.merge(results_df, athletes_df, how='left', on='athlete_full_name')

# Exclure les colonnes non pertinentes
non_pertinent_columns = ['rank_equal', 'rank_position', 'country_code', 'country_3_letter_code', 'athletes', 'athlete_url_x', 'athlete_url_y', 'value_unit']
combined_df.drop(columns=[col for col in non_pertinent_columns if col in combined_df.columns], inplace=True)

# Remplir les valeurs manquantes
combined_df['country_name'].fillna('Unknown', inplace=True)
combined_df['games_participations'].fillna(combined_df['games_participations'].median(), inplace=True)
combined_df['first_game'].fillna('Unknown', inplace=True)
combined_df['athlete_medals'].fillna('NO_MEDAL', inplace=True)

# Afficher les valeurs uniques dans 'athlete_medals' si la colonne existe
if 'athlete_medals' in combined_df.columns:
    print("Valeurs uniques de 'athlete_medals':")
    print(combined_df['athlete_medals'].unique())

    # Traitement des valeurs dans 'athlete_medals'
    def process_medals(medals):
        if medals == 'NO_MEDAL':
            return 0, 0, 0
        gold_medals = 0
        silver_medals = 0
        bronze_medals = 0
        i = 0
        while i < len(medals):
            if medals[i].isdigit():
                num_medals = int(medals[i])
                i += 1
                if i < len(medals) and medals[i] == 'G':
                    gold_medals += num_medals
                elif i < len(medals) and medals[i] == 'S':
                    silver_medals += num_medals
                elif i < len(medals) and medals[i] == 'B':
                    bronze_medals += num_medals
                i += 1
            else:
                i += 1
        return gold_medals, silver_medals, bronze_medals

    combined_df['gold_medals'], combined_df['silver_medals'], combined_df['bronze_medals'] = zip(*combined_df['athlete_medals'].apply(process_medals))
    combined_df = combined_df.drop(columns='athlete_medals')

# Ajouter une colonne pour le nombre total de médailles remportées par chaque athlète
combined_df['total_medals'] = combined_df['gold_medals'] + combined_df['silver_medals'] + combined_df['bronze_medals']

# Afficher un aperçu des données après traitement des médailles
print(combined_df[['athlete_full_name', 'gold_medals', 'silver_medals', 'bronze_medals', 'total_medals']].head())

# Afficher les valeurs uniques de chaque colonne d'intérêt dans combined_df
columns_of_interest = ['discipline_title', 'event_title', 'game_slug', 'participant_type', 'country_name', 
                       'game_year', 'value_unit_category', 'value_type', 'gold_medals', 'silver_medals', 
                       'bronze_medals', 'games_participations', 'first_game', 'medal_type']

for column in columns_of_interest:
    unique_values = combined_df[column].unique()
    print(f"\nValeurs uniques de la colonne '{column}':")
    print(unique_values)

In [ ]:
missing_values_mask = combined_df['athlete_full_name'].isna()
missing_values_df = combined_df[missing_values_mask]
print(missing_values_df.head(10))


In [ ]:
# Sélectionner les colonnes à inclure dans le DataFrame fusionné
columns_to_keep = ['game_slug', 'medal_type', 'country_name', 'value_type', 'game_year', 'value_unit_category']

# Fusionner les lignes avec participant_type == 'GameTeam'
grouped_df = combined_df[combined_df['participant_type'] == 'GameTeam'][columns_to_keep].groupby(['game_slug', 'medal_type']).first().reset_index()

# Ajouter les colonnes de médailles au DataFrame fusionné
grouped_df['gold_medals'] = grouped_df['medal_type'] == 'GOLD'
grouped_df['silver_medals'] = grouped_df['medal_type'] == 'SILVER'
grouped_df['bronze_medals'] = grouped_df['medal_type'] == 'BRONZE'
grouped_df['total_medals'] = grouped_df[['gold_medals', 'silver_medals', 'bronze_medals']].sum(axis=1)

# Afficher un aperçu des données fusionnées
print(grouped_df)


In [ ]:
# Remplacer les valeurs manquantes dans la colonne 'athlete_full_name' pour les lignes avec participant_type == 'GameTeam'
combined_df.loc[combined_df['participant_type'] == 'GameTeam', 'athlete_full_name'] = combined_df['country_name'].apply(lambda x: 'Team ' + x)

# Afficher un aperçu des données après traitement des médailles et des équipes
print(combined_df[['athlete_full_name', 'gold_medals', 'silver_medals', 'bronze_medals', 'total_medals']].head())


In [ ]:
# Afficher les valeurs uniques de la colonne 'athlete_full_name' et leur nombre de lignes correspondantes
athlete_counts = combined_df['athlete_full_name'].value_counts()
print(athlete_counts)


In [ ]:
combined_df


In [ ]:
# Enregistrer le DataFrame dans la base de données SQLite
combined_df.to_sql('cleaned_olympic_results1', con=engine, if_exists='replace', index=False)

In [ ]:
# Afficher le nombre de lignes et de colonnes du DataFrame
print("Nombre de lignes :", len(combined_df))
print("Nombre de colonnes :", len(combined_df.columns))

In [ ]:
# Séparer les données de 2020 des autres années
data_2020 = combined_df[combined_df['game_year'] == 2020]
data_other_years = combined_df[combined_df['game_year'] != 2020]

# Afficher un aperçu des données de 2020
print("Aperçu des données de 2020:")
print(data_2020.head())

# Afficher un aperçu des données des autres années
print("Aperçu des données des autres années:")
print(data_other_years.head())


In [ ]:
# Sélectionner les fonctionnalités pertinentes pour l'entraînement du modèle
features = ['discipline_title', 'event_title', 'game_slug', 'participant_type', 'country_name', 
            'game_year', 'value_unit_category', 'value_type', 'gold_medals', 'silver_medals', 'bronze_medals', 
            'games_participations', 'first_game']

X_train = data_other_years[features]
y_train = data_other_years['medal_type']

# Gérer les valeurs manquantes dans les fonctionnalités
X_train['value_unit_category'].fillna('NaN', inplace=True)
X_train['first_game'].fillna('Unknown', inplace=True)

# Pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['game_year', 'games_participations', 'gold_medals', 'silver_medals', 'bronze_medals']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['discipline_title', 'event_title', 'game_slug', 
                                                         'participant_type', 'country_name', 'value_unit_category', 
                                                         'value_type', 'first_game'])
    ])

# Pipeline de modèle
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Entraîner le modèle sur les données des années précédentes
model.fit(X_train, y_train)


In [ ]:
# Préparer les données de 2020 pour la prédiction
X_2020 = data_2020[features]

# Gérer les valeurs manquantes dans les fonctionnalités
X_2020['value_unit_category'].fillna('NaN', inplace=True)
X_2020['first_game'].fillna('Unknown', inplace=True)

# Faire des prédictions sur les données de 2020
y_pred_2020 = model.predict(X_2020)

# Ajouter les prédictions au DataFrame de 2020
data_2020['predicted_medal_type'] = y_pred_2020

# Afficher un aperçu des prédictions de 2020
print("Aperçu des prédictions pour 2020:")
print(data_2020[['athlete_full_name', 'predicted_medal_type']].head())


In [ ]:
# Comparer les prédictions aux résultats réels
comparison = data_2020[['athlete_full_name', 'medal_type', 'predicted_medal_type']]

# Calculer la précision des prédictions
accuracy = (comparison['medal_type'] == comparison['predicted_medal_type']).mean()
print(f"Précision des prédictions pour 2020: {accuracy:.2f}")

# Afficher les prédictions incorrectes
incorrect_predictions = comparison[comparison['medal_type'] != comparison['predicted_medal_type']]
print("Prédictions incorrectes:")
print(incorrect_predictions.head())


In [ ]:
# Analyser les prédictions incorrectes
incorrect_predictions = comparison[comparison['medal_type'] != comparison['predicted_medal_type']]

# Afficher un aperçu des prédictions incorrectes
print("Prédictions incorrectes:")
print(incorrect_predictions.head())

# Examiner les caractéristiques des prédictions incorrectes
incorrect_details = data_2020[data_2020['athlete_full_name'].isin(incorrect_predictions['athlete_full_name'])]
print("Détails des prédictions incorrectes:")
print(incorrect_details.head(10))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Visualiser la répartition des prédictions correctes et incorrectes
plt.figure(figsize=(10, 6))
sns.countplot(x='predicted_medal_type', hue='medal_type', data=data_2020)
plt.title('Comparaison des Prédictions et des Résultats Réels pour 2020')
plt.xlabel('Type de Médaille Prédite')
plt.ylabel('Nombre')
plt.legend(title='Type de Médaille Réel')
plt.show()


In [ ]:
import joblib

# Sauvegarder le modèle dans un fichier
model_filename = 'olympic_model.pkl'
joblib.dump(model, model_filename)
print(f"Modèle sauvegardé dans le fichier '{model_filename}'")


In [ ]:
print("Valeurs uniques de 'medal_type' après remplacement des None:")
print(results_df['medal_type'].unique())

In [ ]:
# Sélectionner la cible
y = results_df['medal_type']

# Gérer les valeurs manquantes
X['value_unit_category'].fillna('NaN', inplace=True)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['game_year']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['discipline_title', 'event_title', 'game_slug', 'participant_type', 'country_name', 'value_unit_category', 'value_type'])
    ])

# Pipeline de modèle
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Entraîner le modèle
model.fit(X_train, y_train)

# Prédire sur les données de test
y_pred = model.predict(X_test)

# Évaluer le modèle
print("\nRapport de classification:")
print(classification_report(y_test, y_pred))

# Exemple de prédiction
example = X_test.iloc[0:1]  # Convertir en DataFrame avec une seule ligne
print("\nExemple de prédiction pour l'entrée suivante:")
print(example)
print("Prédiction du modèle:", model.predict(example))

le modèle est une classification du type de médaille que pourrait remporter un athlète (ou une équipe) dans une certaine épreuve des Jeux Olympiques. Le modèle prédit parmi les catégories suivantes : 'GOLD', 'SILVER', 'BRONZE', ou 'NO_MEDAL'.

In [ ]:
import joblib

joblib.dump(model, 'model.pkl')

In [ ]:
# Charger les colonnes spécifiées de la table olympic_athletes
athletes_df = pd.read_sql("SELECT athlete_url, athlete_full_name, games_participations, first_game, athlete_medals FROM olympic_athletes;", con=engine)

# Afficher un aperçu des données chargées
print(athletes_df.head())

In [ ]:
# Remplacer les valeurs None dans 'athlete_medals' par 'NO_MEDAL'
athletes_df['athlete_medals'].fillna('NO_MEDAL', inplace=True)

# Afficher un aperçu des données après remplacement
print(athletes_df.head())

In [ ]:
# Joindre les datasets results_df et athletes_df sur 'athlete_full_name' ou une autre clé appropriée
combined_df = pd.merge(results_df, athletes_df, how='left', left_on='athlete_full_name', right_on='athlete_full_name')

# Sélectionner les fonctionnalités pertinentes
features = ['discipline_title', 'event_title', 'game_slug', 'participant_type', 'country_name', 'game_year', 'value_unit_category', 'value_type', 'athlete_medals']
X = combined_df[features]

# Sélectionner la cible
y = combined_df['medal_type']

# Gérer les valeurs manquantes
X['value_unit_category'].fillna('NaN', inplace=True)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['game_year']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['discipline_title', 'event_title', 'game_slug', 'participant_type', 'country_name', 'value_unit_category', 'value_type', 'athlete_medals'])
    ])

# Pipeline de modèle
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Entraîner le modèle
model.fit(X_train, y_train)

# Prédire sur les données de test
y_pred = model.predict(X_test)

# Évaluer le modèle
print("\nRapport de classification:")
print(classification_report(y_test, y_pred))

# Exemple de prédiction
example = X_test.iloc[0:1]  # Convertir en DataFrame avec une seule ligne
print("\nExemple de prédiction pour l'entrée suivante:")
print(example)
print("Prédiction du modèle:", model.predict(example))

In [ ]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# Charger les données existantes pour vérifier
existing_data = pd.read_sql("SELECT * FROM cleaned_olympic_data;", con=engine)
print("Aperçu des données existantes dans la table cleaned_olympic_data:")
print(existing_data.head())

print("Colonnes de athletes_df:", athletes_df.columns)
print("Colonnes de cleaned_olympic_data:", existing_data.columns)

In [ ]:
# Afficher un aperçu des données après remplacement
print("Aperçu des données de athletes_df après remplacement des None:")
print(athletes_df.head())


In [ ]:
# Vérifier les colonnes pour s'assurer de la compatibilité
print("Colonnes de athletes_df:", athletes_df.columns)
print("Colonnes de cleaned_olympic_data:", existing_data.columns)

In [ ]:
# Charger les colonnes spécifiées de la table olympic_athletes
athletes_df = pd.read_sql("SELECT athlete_url, athlete_full_name, games_participations, first_game, athlete_medals FROM olympic_athletes;", con=engine)
athletes_df

In [ ]:
# Remplacer les valeurs None dans 'athlete_medals' par 'NO_MEDAL'
athletes_df['athlete_medals'].fillna('NO_MEDAL', inplace=True)
athletes_df

In [ ]:
unique_medal_values = athletes_df['athlete_medals'].unique()
print("Valeurs uniques dans la colonne 'athlete_medals':")
print(unique_medal_values)

In [ ]:
import re

# Fonction pour extraire les médailles
def extract_medals(medal_str):
    medals = {'G': 0, 'S': 0, 'B': 0}
    if medal_str != 'NO_MEDAL':
        matches = re.findall(r'(\d+)([GSB])', medal_str)
        for count, medal_type in matches:
            medals[medal_type] += int(count)
    return medals

# Appliquer la fonction d'extraction pour créer des colonnes pour chaque type de médaille
athletes_df['gold_medals'] = athletes_df['athlete_medals'].apply(lambda x: extract_medals(x)['G'])
athletes_df['silver_medals'] = athletes_df['athlete_medals'].apply(lambda x: extract_medals(x)['S'])
athletes_df['bronze_medals'] = athletes_df['athlete_medals'].apply(lambda x: extract_medals(x)['B'])

# Afficher un aperçu des données après transformation
print("Aperçu des données de athletes_df après transformation des médailles:")
print(athletes_df.head())

In [ ]:

# Charger les données existantes de results_df
results_df = pd.read_sql("SELECT * FROM olympic_results;", con=engine)

# Joindre les datasets results_df et athletes_df sur les clés appropriées
combined_df = pd.merge(results_df, athletes_df[['athlete_url', 'athlete_full_name', 'games_participations', 'first_game', 'gold_medals', 'silver_medals', 'bronze_medals']],
                       left_on='athlete_url', right_on='athlete_url', how='left')

# Remplacer les valeurs non numériques par des zéros dans 'games_participations'
combined_df['games_participations'] = combined_df['games_participations'].fillna(0).astype(int)

# Calculer les nouvelles statistiques par pays et année
updated_data = combined_df.groupby(['country_name', 'game_year']).agg(
    BRONZE=('medal_type', lambda x: (x == 'BRONZE').sum()),
    GOLD=('medal_type', lambda x: (x == 'GOLD').sum()),
    SILVER=('medal_type', lambda x: (x == 'SILVER').sum()),
    total_medals=('medal_type', lambda x: x.isin(['BRONZE', 'GOLD', 'SILVER']).sum()),
    total_disciplines=('discipline_title', 'nunique'),
    total_events=('event_title', 'nunique'),
    athlete_gold_medals=('gold_medals', 'sum'),
    athlete_silver_medals=('silver_medals', 'sum'),
    athlete_bronze_medals=('bronze_medals', 'sum')
).reset_index()


In [ ]:
# Charger les données existantes de results_df
results_df = pd.read_sql("SELECT * FROM olympic_results;", con=engine)

# Extraire l'année des jeux à partir de la colonne 'slug_game'
results_df['game_year'] = results_df['slug_game'].apply(lambda x: int(x.split('-')[-1]))

# Vérifier l'ajout de la colonne 'game_year'
print("Colonnes disponibles dans results_df après extraction de l'année:")
print(results_df.columns)

print("\nPremières lignes de results_df après extraction de l'année:")
print(results_df.head())

In [ ]:
# Charger les données existantes de results_df avec la colonne 'game_year' ajoutée
results_df = pd.read_sql("SELECT * FROM olympic_results;", con=engine)
results_df['game_year'] = results_df['slug_game'].apply(lambda x: int(x.split('-')[-1]))

# Joindre les datasets results_df et athletes_df sur les clés appropriées
combined_df = pd.merge(results_df, athletes_df[['athlete_url', 'athlete_full_name', 'games_participations', 'first_game', 'gold_medals', 'silver_medals', 'bronze_medals']],
                       left_on='athlete_url', right_on='athlete_url', how='left')

# Remplacer les valeurs non numériques par des zéros dans 'games_participations'
combined_df['games_participations'] = combined_df['games_participations'].fillna(0).astype(int)

# Calculer les nouvelles statistiques par pays et année
updated_data = combined_df.groupby(['country_name', 'game_year']).agg(
    BRONZE=('medal_type', lambda x: (x == 'BRONZE').sum()),
    GOLD=('medal_type', lambda x: (x == 'GOLD').sum()),
    SILVER=('medal_type', lambda x: (x == 'SILVER').sum()),
    total_medals=('medal_type', lambda x: x.isin(['BRONZE', 'GOLD', 'SILVER']).sum()),
    total_disciplines=('discipline_title', 'nunique'),
    total_events=('event_title', 'nunique'),
    athlete_gold_medals=('gold_medals', 'sum'),
    athlete_silver_medals=('silver_medals', 'sum'),
    athlete_bronze_medals=('bronze_medals', 'sum')
).reset_index()

# Sauvegarder les nouvelles données dans la base de données
updated_data.to_sql('cleaned_olympic_data', con=engine, if_exists='replace', index=False)
print("Les données combinées ont été sauvegardées dans la base de données.")

# Vérifier les données après ajout
updated_data_check = pd.read_sql("SELECT * FROM cleaned_olympic_data;", con=engine)
print("Aperçu des données mises à jour dans la table cleaned_olympic_data:")
print(updated_data_check.tail())


In [ ]:
unique_countries = results_df['country_name'].unique()
print("Valeurs uniques dans la colonne 'country_name':")
print(unique_countries)

In [ ]:
# Nettoyage des anomalies
def clean_country_name(country_name):
    if pd.isna(country_name):
        return None
    # Supprimer les URLs
    country_name = re.sub(r'https?://\S+', '', country_name)
    # Supprimer les espaces blancs en début et fin
    country_name = country_name.strip()
    # Remplacer par les noms de pays corrects
    return country_renaming.get(country_name, country_name)

results_df['country_name'] = results_df['country_name'].apply(clean_country_name)
results_df['country_name'] = results_df['country_name'].replace({
    'None)]"': None, '10': None, '25': None, '17': None, '14': None, '5': None, '3': None, '11': None, '1': None, '16': None, '': None, "Jr.'": None
})

# Vérifier les valeurs uniques après nettoyage
cleaned_unique_countries = results_df['country_name'].unique()
print("Valeurs uniques dans la colonne 'country_name' après nettoyage:")
print(cleaned_unique_countries)

# Sauvegarder les données nettoyées dans la base de données
results_df.to_sql('cleaned_olympic_results', con=engine, if_exists='replace', index=False)
print("Les données nettoyées ont été sauvegardées dans la base de données.")

# Vérifier les données après sauvegarde
updated_data_check = pd.read_sql("SELECT * FROM cleaned_olympic_results;", con=engine)
print("Aperçu des données mises à jour dans la table cleaned_olympic_results:")
print(updated_data_check.head())

In [ ]:
print(updated_data_check.head())